In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import re
import random
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
from tqdm import tqdm
from glob import glob
warnings.filterwarnings(action='ignore')


In [2]:
path = 'D:/python_project/hyunmin_project/운동동작분류/'

train=pd.read_csv(path + 'open/train_features.csv')
train_label=pd.read_csv(path + 'open/train_labels.csv')
test=pd.read_csv(path + 'open/test_features.csv')
submission=pd.read_csv(path + 'open/sample_submission.csv')

In [3]:
train.shape, train_label.shape, test.shape, submission.shape

((1875000, 8), (3125, 3), (469200, 8), (782, 62))

In [4]:
train.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234


In [41]:
train_id = train['id'].unique()

In [42]:
train[['id', 'time']].groupby('id').count()

,time
id,
0,600
1,600
2,600
3,600
4,600
...,...
3120,600
3121,600
3122,600


In [43]:
train_label

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [44]:
train_label['label'].value_counts().tail(5)

53    13
52    12
13    12
12    12
0     12
Name: label, dtype: int64

# Non-Exercise를 제외하고 따로 분류

In [45]:
train_Non_id = train_label['id'][~train_label['label'].isin([26])]
train_Non_id.unique()
exercise_id = list(train_Non_id.unique())
len(exercise_id)


1607

In [46]:
train_ex = train[train['id'].isin(exercise_id)]
train_ex.shape

(964200, 8)

# 데이터 증강

In [47]:
train['id'].unique()

array([   0,    1,    2, ..., 3122, 3123, 3124], dtype=int64)

In [48]:
x_aug_train = []

for uid in tqdm(train['id'].unique()):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32).T
    x_aug_train.append(temp)

x_aug_train = np.array(x_aug_train, np.float32)
x_aug_train = x_aug_train[:,:,:,np.newaxis]

100%|██████████████████████████████████████████████████████████████████████████| 3125/3125 [00:08<00:00, 351.63it/s]


In [88]:
def aug(data, id_list, shift = 0):
    shift_data = np.roll(data, shift, axis=2)
    aug_list = []
    for i in id_list:
        agg_list = []
        for j in range(3):
            agg_list.append(shift_data[i][j,:,0].max())
            agg_list.append(shift_data[i][j,:,0].min())
            agg_list.append(shift_data[i][j,:,0].mean())
        aug_list.append(agg_list)
    
    aug_list = np.array(aug_list)
    print(aug_list.shape)
    return aug_list

In [89]:
aug_data = []
for _ in range(5):
    data = aug(x_aug_train, train_id, int(random.random()*600))
    aug_data += list(data)


(3125, 9)
(3125, 9)
(3125, 9)
(3125, 9)
(3125, 9)


In [90]:
aug_data[0]

array([ 1.3442676 ,  0.5919403 ,  0.93132895,  0.17687133, -0.6241134 ,
       -0.21847117, -0.05487625, -0.78633565, -0.37042156], dtype=float32)

In [91]:
def plot_acc_id(df, id_num):
    return df.query('id == @id_num')[['time', 'acc_x', 'acc_y', 'acc_z']].plot(x = 'time')

def plot_gy_id(df, id_num):
    return df.query('id == @id_num')[['time', 'gy_x', 'gy_y', 'gy_z']].plot(x = 'time')

In [92]:
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
X_train = train[features].groupby('id').agg(['max', 'min', 'mean'])
#Ex_train = train_ex[features].groupby('id').agg(['max', 'min', 'mean', 'std', 'var'])
X_test = test[features].groupby('id').agg(['max', 'min', 'mean'])

In [93]:
X_train

acc_x                         acc_y                         acc_z  \
           max       min      mean       max       min      mean       max   
id                                                                           
0     1.344268  0.591940  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1     1.234020 -2.156208 -0.766580  0.700065 -1.295598 -0.317258  0.888661   
2     1.219836 -1.142847  0.039836  0.650645 -0.690990 -0.082403  1.332992   
3    -0.622250 -1.417751 -0.887702  0.283721 -0.540827 -0.087668  0.598814   
4     0.599720 -2.429109 -0.659018  1.724782 -2.055076 -0.337067  2.678034   
...        ...       ...       ...       ...       ...       ...       ...   
3120  0.390798 -1.624711 -0.300454  0.168070 -1.289257 -0.669209  0.849966   
3121 -0.446650 -1.575455 -0.974298  0.117965 -0.609743 -0.233373  0.218621   
3122  0.744666 -2.578974 -1.114246  1.268138 -2.036646 -0.362196  0.803950   
3123  0.915846 -0.929133 -0.111333  1.473727  0.272406  0.880362  0.382916   
3124  0.538809 -1.013813 -0.434048 -0.193247 -0.952761 -0.623010  0.543977   

                          
           min      mean  
id                        
0    -0.786336 -0.370422  
1    -1.019531 -0.004223  
2     0.073846  0.626012  
3    -0.257124  0.227357  
4    -1.250483  0.202758  
...        ...       ...  
3120 -0.551185  0.335934  
3121 -0.403201 -0.073771  
3122 -0.459267  0.241518  
3123 -0.646863 -0.122757  
3124 -0.380657  0.226848  

[3125 rows x 9 columns]

In [94]:
# train_100 = train[ (0<=train['time']) & (train['time']<=100) ]
# train_200 = train[ (100<=train['time']) & (train['time']<=200) ]
# train_300 = train[ (200<=train['time']) & (train['time']<=300) ]
# train_400 = train[ (300<=train['time']) & (train['time']<=400) ]
# train_500 = train[ (400<=train['time']) & (train['time']<=500) ]
# train_600 = train[ (500<=train['time']) & (train['time']<=600) ]

In [95]:
# X_train_100 = train_100[features].groupby('id').agg(['max', 'min', 'mean'])
# X_train_200 = train_200[features].groupby('id').agg(['max', 'min', 'mean'])
# X_train_300 = train_300[features].groupby('id').agg(['max', 'min', 'mean'])
# X_train_400 = train_400[features].groupby('id').agg(['max', 'min', 'mean'])
# X_train_500 = train_500[features].groupby('id').agg(['max', 'min', 'mean'])
# X_train_600 = train_600[features].groupby('id').agg(['max', 'min', 'mean'])

In [96]:
# X_train = pd.concat([X_train_100, X_train_200, X_train_300, X_train_400, X_train_500, X_train_600], axis=1)

In [97]:
colum_list = []
for i in range(int(X_train.shape[1])):
    colum_list.append(i)

In [98]:
X_train.columns = colum_list
#Ex_train.columns = colum_list

In [99]:
np.array(X_train).shape, np.array(aug_data).shape

((3125, 9), (15625, 9))

In [100]:
X_train = np.concatenate((np.array(X_train), aug_data), axis=0)

In [101]:
X_train.shape, X_test.shape

((18750, 9), (782, 9))

In [102]:
binary_label = []
for i in train_label['label']:
    if i == 26:
        binary_label.append(0)
    else:
        binary_label.append(1)

multi_label = []
for i in train_label['label']:
    if i != 26:
        multi_label.append(i)


In [103]:
Y_train = []
for i in range(6):
    Y_train += list(train_label['label'])
len(Y_train)

18750

In [104]:
pd.DataFrame(Y_train)[0].value_counts()

26    9108
8      582
28     330
60     288
18     282
      ... 
53      78
52      72
13      72
12      72
0       72
Name: 0, Length: 61, dtype: int64

In [105]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train,
                                                    test_size=0.2, random_state=0)
x_train.shape

(15000, 9)

In [106]:
from sklearn.model_selection import KFold, GridSearchCV
import xgboost as xgb
import lightgbm as lgb
param_test1 = {
 'max_depth':[4,6,8,10],
 'min_child_weight':[1, 3, 5]
}
gsearch1 = GridSearchCV(estimator = lgb.LGBMClassifier(learning_rate=0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  objective= 'multiclass', 
                                                  nthread=-1, 
                                                  eval_metric='logloss',
                                                  scale_pos_weight=1, seed=2019),
                        
param_grid = param_test1, scoring='neg_log_loss',n_jobs=-1,iid=False, cv=5, verbose=10)
gsearch1.fit(x_train, y_train, eval_set=[(x_test, y_test)], early_stopping_rounds=30, 
                  eval_metric="logloss")
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  56 out of  60 | elapsed: 10.2min remaining:   43.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 10.7min finished


[1]	valid_0's multi_logloss: 1.62468
Training until validation scores don't improve for 30 rounds
[2]	valid_0's multi_logloss: 1.25409
[3]	valid_0's multi_logloss: 1.02494
[4]	valid_0's multi_logloss: 0.859354
[5]	valid_0's multi_logloss: 0.732903
[6]	valid_0's multi_logloss: 0.634746
[7]	valid_0's multi_logloss: 0.554123
[8]	valid_0's multi_logloss: 0.486279
[9]	valid_0's multi_logloss: 0.429369
[10]	valid_0's multi_logloss: 0.379622
[11]	valid_0's multi_logloss: 0.337624
[12]	valid_0's multi_logloss: 0.301032
[13]	valid_0's multi_logloss: 0.268851
[14]	valid_0's multi_logloss: 0.241061
[15]	valid_0's multi_logloss: 0.216319
[16]	valid_0's multi_logloss: 0.195666
[17]	valid_0's multi_logloss: 0.176711
[18]	valid_0's multi_logloss: 0.159776
[19]	valid_0's multi_logloss: 0.145502
[20]	valid_0's multi_logloss: 0.132818
[21]	valid_0's multi_logloss: 0.121609
[22]	valid_0's multi_logloss: 0.111512
[23]	valid_0's multi_logloss: 0.102931
[24]	valid_0's multi_logloss: 0.0944987
[25]	valid_0's

[205]	valid_0's multi_logloss: 0.00770659
[206]	valid_0's multi_logloss: 0.00768982
[207]	valid_0's multi_logloss: 0.00766405
[208]	valid_0's multi_logloss: 0.00764629
[209]	valid_0's multi_logloss: 0.00763391
[210]	valid_0's multi_logloss: 0.00761771
[211]	valid_0's multi_logloss: 0.00759467
[212]	valid_0's multi_logloss: 0.00757834
[213]	valid_0's multi_logloss: 0.00755869
[214]	valid_0's multi_logloss: 0.00753435
[215]	valid_0's multi_logloss: 0.00752228
[216]	valid_0's multi_logloss: 0.00751101
[217]	valid_0's multi_logloss: 0.00749221
[218]	valid_0's multi_logloss: 0.00746934
[219]	valid_0's multi_logloss: 0.00745291
[220]	valid_0's multi_logloss: 0.00743525
[221]	valid_0's multi_logloss: 0.00741467
[222]	valid_0's multi_logloss: 0.00739444
[223]	valid_0's multi_logloss: 0.00737899
[224]	valid_0's multi_logloss: 0.00735343
[225]	valid_0's multi_logloss: 0.00733317
[226]	valid_0's multi_logloss: 0.00731055
[227]	valid_0's multi_logloss: 0.00729365
[228]	valid_0's multi_logloss: 0.0

[411]	valid_0's multi_logloss: 0.00542871
[412]	valid_0's multi_logloss: 0.00542149
[413]	valid_0's multi_logloss: 0.00541332
[414]	valid_0's multi_logloss: 0.00540714
[415]	valid_0's multi_logloss: 0.00539889
[416]	valid_0's multi_logloss: 0.00539402
[417]	valid_0's multi_logloss: 0.005389
[418]	valid_0's multi_logloss: 0.00538443
[419]	valid_0's multi_logloss: 0.00537986
[420]	valid_0's multi_logloss: 0.00537189
[421]	valid_0's multi_logloss: 0.00536955
[422]	valid_0's multi_logloss: 0.00535868
[423]	valid_0's multi_logloss: 0.00535648
[424]	valid_0's multi_logloss: 0.00535111
[425]	valid_0's multi_logloss: 0.00534774
[426]	valid_0's multi_logloss: 0.00533695
[427]	valid_0's multi_logloss: 0.00532925
[428]	valid_0's multi_logloss: 0.00532524
[429]	valid_0's multi_logloss: 0.00531972
[430]	valid_0's multi_logloss: 0.00531323
[431]	valid_0's multi_logloss: 0.0053045
[432]	valid_0's multi_logloss: 0.00530101
[433]	valid_0's multi_logloss: 0.00529841
[434]	valid_0's multi_logloss: 0.0052

[616]	valid_0's multi_logloss: 0.00465782
[617]	valid_0's multi_logloss: 0.00465528
[618]	valid_0's multi_logloss: 0.00465221
[619]	valid_0's multi_logloss: 0.00464958
[620]	valid_0's multi_logloss: 0.00464876
[621]	valid_0's multi_logloss: 0.00464664
[622]	valid_0's multi_logloss: 0.00464382
[623]	valid_0's multi_logloss: 0.00464121
[624]	valid_0's multi_logloss: 0.00463921
[625]	valid_0's multi_logloss: 0.0046368
[626]	valid_0's multi_logloss: 0.00463478
[627]	valid_0's multi_logloss: 0.00463175
[628]	valid_0's multi_logloss: 0.00462961
[629]	valid_0's multi_logloss: 0.00462569
[630]	valid_0's multi_logloss: 0.00462284
[631]	valid_0's multi_logloss: 0.00461912
[632]	valid_0's multi_logloss: 0.00461688
[633]	valid_0's multi_logloss: 0.00461242
[634]	valid_0's multi_logloss: 0.00461005
[635]	valid_0's multi_logloss: 0.00460778
[636]	valid_0's multi_logloss: 0.00460577
[637]	valid_0's multi_logloss: 0.0046035
[638]	valid_0's multi_logloss: 0.00459993
[639]	valid_0's multi_logloss: 0.004

[821]	valid_0's multi_logloss: 0.00428471
[822]	valid_0's multi_logloss: 0.00428254
[823]	valid_0's multi_logloss: 0.00428097
[824]	valid_0's multi_logloss: 0.00427975
[825]	valid_0's multi_logloss: 0.0042782
[826]	valid_0's multi_logloss: 0.0042778
[827]	valid_0's multi_logloss: 0.00427738
[828]	valid_0's multi_logloss: 0.00427603
[829]	valid_0's multi_logloss: 0.00427531
[830]	valid_0's multi_logloss: 0.00427421
[831]	valid_0's multi_logloss: 0.00427309
[832]	valid_0's multi_logloss: 0.00427203
[833]	valid_0's multi_logloss: 0.00427022
[834]	valid_0's multi_logloss: 0.00426992
[835]	valid_0's multi_logloss: 0.00426875
[836]	valid_0's multi_logloss: 0.00426809
[837]	valid_0's multi_logloss: 0.00426654
[838]	valid_0's multi_logloss: 0.00426482
[839]	valid_0's multi_logloss: 0.00426321
[840]	valid_0's multi_logloss: 0.00426125
[841]	valid_0's multi_logloss: 0.00425912
[842]	valid_0's multi_logloss: 0.00425833
[843]	valid_0's multi_logloss: 0.00425746
[844]	valid_0's multi_logloss: 0.004

({'mean_fit_time': array([34.26522999, 47.41003723, 49.7537704 , 37.42674356, 44.98109651,
         53.05618205, 33.43247828, 42.0416626 , 48.79571843, 29.87911949,
         40.94373302, 47.60689321]),
  'std_fit_time': array([ 8.08889616,  5.48129516,  5.06298403,  8.49037633,  4.00860786,
          3.34656161, 12.03687326,  5.33454021,  1.6215588 , 11.76039425,
          5.89858429,  0.87621777]),
  'mean_score_time': array([21.36486888, 25.04284949, 27.05966253, 16.91320596, 20.39768124,
         23.93735857, 13.17398582, 18.06215396, 24.03843207, 12.4953907 ,
         17.5825953 , 19.829423  ]),
  'std_score_time': array([7.9150157 , 4.76616265, 3.10502882, 5.2869951 , 4.13747972,
         3.43435614, 2.74914412, 2.84929327, 1.92839754, 2.96407725,
         2.47141497, 2.89875384]),
  'param_max_depth': masked_array(data=[4, 4, 4, 6, 6, 6, 8, 8, 8, 10, 10, 10],
               mask=[False, False, False, False, False, False, False, False,
                     False, False, False, Fal

In [122]:
m_y_train = multi_label
len(m_y_train)

1607

In [123]:
from sklearn.model_selection import train_test_split
ex_train, ex_test, ey_train, ey_test = train_test_split(Ex_train, m_y_train,
                                                    test_size=0.2, random_state=0)

NameError: name 'Ex_train' is not defined

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV
import xgboost as xgb
param_test1 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch2 = GridSearchCV(estimator = lgb.LGBMClassifier(learning_rate=0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=4, 
                                                  min_child_weight=3, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.6,
                                                  objective= 'multiclass', 
                                                  nthread=-1, 
                                                  eval_metric='logloss',
                                                  scale_pos_weight=1, seed=2019),
                        
param_grid = param_test1, scoring='neg_log_loss',n_jobs=-1,iid=False, cv=5, verbose=10)
gsearch1.fit(ex_train, ey_train, eval_set=[(ex_test, ey_test)], early_stopping_rounds=30, 
                  eval_metric="logloss")
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

In [132]:
from sklearn.model_selection import KFold, GridSearchCV
import xgboost as xgb
param_test1 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=6, 
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.9, 
                                                  colsample_bytree=0.6,
                                                  objective= 'multi:softmax', 
                                                  nthread=-1, 
                                                  eval_metric='mlogloss',
                                                  scale_pos_weight=1, seed=2019),
                        
param_grid = param_test1, scoring='neg_log_loss',n_jobs=-1,iid=False, cv=5, verbose=10)
gsearch3.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=50, 
                  eval_metric="mlogloss")
gsearch3.cv_results_, gsearch2.best_params_, gsearch2.best_score_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:  2.0min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:  2.0min remaining:   50.5s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  2.0min remaining:   20.7s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.0min remaining:    0.0s


KeyboardInterrupt: 

In [107]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
model=[]
Y_train = np.array(Y_train)
for train_idx, val_idx in k_fold.split(X_train):

    # split train, validation set
    X = X_train[train_idx]
    y = Y_train[train_idx]
    valid_x = X_train[val_idx]
    valid_y = Y_train[val_idx]

    lgb_model = lgb.LGBMClassifier(learning_rate=0.05, 
                                                  n_estimators=1000, 
                                                  max_depth=10, 
                                                  min_child_weight=5, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.6,
                                                  objective= 'multiclass', 
                                                  nthread=-1, 
                                                  eval_metric='logloss',
                                                  scale_pos_weight=1, seed=2019)
    
    lgb_model.fit(X, y, eval_set=[(valid_x, valid_y)], early_stopping_rounds=50, 
                  eval_metric="logloss")
    
    model.append(lgb_model)

[1]	valid_0's multi_logloss: 2.49616
Training until validation scores don't improve for 50 rounds
[2]	valid_0's multi_logloss: 2.28724
[3]	valid_0's multi_logloss: 2.11488
[4]	valid_0's multi_logloss: 1.96113
[5]	valid_0's multi_logloss: 1.83037
[6]	valid_0's multi_logloss: 1.71758
[7]	valid_0's multi_logloss: 1.61467
[8]	valid_0's multi_logloss: 1.52304
[9]	valid_0's multi_logloss: 1.43994
[10]	valid_0's multi_logloss: 1.36614
[11]	valid_0's multi_logloss: 1.2988
[12]	valid_0's multi_logloss: 1.23632
[13]	valid_0's multi_logloss: 1.17955
[14]	valid_0's multi_logloss: 1.12779
[15]	valid_0's multi_logloss: 1.07803
[16]	valid_0's multi_logloss: 1.03174
[17]	valid_0's multi_logloss: 0.989751
[18]	valid_0's multi_logloss: 0.949354
[19]	valid_0's multi_logloss: 0.912971
[20]	valid_0's multi_logloss: 0.877841
[21]	valid_0's multi_logloss: 0.844701
[22]	valid_0's multi_logloss: 0.812041
[23]	valid_0's multi_logloss: 0.781884
[24]	valid_0's multi_logloss: 0.753579
[25]	valid_0's multi_logloss:

[215]	valid_0's multi_logloss: 0.0742014
[216]	valid_0's multi_logloss: 0.0738685
[217]	valid_0's multi_logloss: 0.0735897
[218]	valid_0's multi_logloss: 0.0734277
[219]	valid_0's multi_logloss: 0.0731148
[220]	valid_0's multi_logloss: 0.0728401
[221]	valid_0's multi_logloss: 0.0725692
[222]	valid_0's multi_logloss: 0.0723403
[223]	valid_0's multi_logloss: 0.0721451
[224]	valid_0's multi_logloss: 0.0718724
[225]	valid_0's multi_logloss: 0.0716152
[226]	valid_0's multi_logloss: 0.0713658
[227]	valid_0's multi_logloss: 0.0711066
[228]	valid_0's multi_logloss: 0.0708995
[229]	valid_0's multi_logloss: 0.0706084
[230]	valid_0's multi_logloss: 0.0702861
[231]	valid_0's multi_logloss: 0.0700419
[232]	valid_0's multi_logloss: 0.0698034
[233]	valid_0's multi_logloss: 0.0695601
[234]	valid_0's multi_logloss: 0.0692692
[235]	valid_0's multi_logloss: 0.0691062
[236]	valid_0's multi_logloss: 0.0688454
[237]	valid_0's multi_logloss: 0.0685614
[238]	valid_0's multi_logloss: 0.0683251
[239]	valid_0's 

[422]	valid_0's multi_logloss: 0.0471666
[423]	valid_0's multi_logloss: 0.0471122
[424]	valid_0's multi_logloss: 0.0470571
[425]	valid_0's multi_logloss: 0.0469798
[426]	valid_0's multi_logloss: 0.0469211
[427]	valid_0's multi_logloss: 0.0468743
[428]	valid_0's multi_logloss: 0.0468048
[429]	valid_0's multi_logloss: 0.0467418
[430]	valid_0's multi_logloss: 0.0466643
[431]	valid_0's multi_logloss: 0.0466234
[432]	valid_0's multi_logloss: 0.0465565
[433]	valid_0's multi_logloss: 0.0464919
[434]	valid_0's multi_logloss: 0.046421
[435]	valid_0's multi_logloss: 0.0463801
[436]	valid_0's multi_logloss: 0.0463427
[437]	valid_0's multi_logloss: 0.046299
[438]	valid_0's multi_logloss: 0.0462329
[439]	valid_0's multi_logloss: 0.0461912
[440]	valid_0's multi_logloss: 0.0461166
[441]	valid_0's multi_logloss: 0.0460862
[442]	valid_0's multi_logloss: 0.0460082
[443]	valid_0's multi_logloss: 0.0459728
[444]	valid_0's multi_logloss: 0.045928
[445]	valid_0's multi_logloss: 0.0458873
[446]	valid_0's mul

[635]	valid_0's multi_logloss: 0.0387005
[636]	valid_0's multi_logloss: 0.0386742
[637]	valid_0's multi_logloss: 0.0386532
[638]	valid_0's multi_logloss: 0.038639
[639]	valid_0's multi_logloss: 0.0386139
[640]	valid_0's multi_logloss: 0.0385899
[641]	valid_0's multi_logloss: 0.0385729
[642]	valid_0's multi_logloss: 0.0385444
[643]	valid_0's multi_logloss: 0.0385189
[644]	valid_0's multi_logloss: 0.0384959
[645]	valid_0's multi_logloss: 0.0384755
[646]	valid_0's multi_logloss: 0.0384431
[647]	valid_0's multi_logloss: 0.038409
[648]	valid_0's multi_logloss: 0.0383834
[649]	valid_0's multi_logloss: 0.0383592
[650]	valid_0's multi_logloss: 0.0383331
[651]	valid_0's multi_logloss: 0.0382958
[652]	valid_0's multi_logloss: 0.0382754
[653]	valid_0's multi_logloss: 0.0382397
[654]	valid_0's multi_logloss: 0.0382156
[655]	valid_0's multi_logloss: 0.0381917
[656]	valid_0's multi_logloss: 0.0381673
[657]	valid_0's multi_logloss: 0.0381345
[658]	valid_0's multi_logloss: 0.0381079
[659]	valid_0's mu

[850]	valid_0's multi_logloss: 0.0347167
[851]	valid_0's multi_logloss: 0.0347038
[852]	valid_0's multi_logloss: 0.034696
[853]	valid_0's multi_logloss: 0.034672
[854]	valid_0's multi_logloss: 0.0346558
[855]	valid_0's multi_logloss: 0.0346431
[856]	valid_0's multi_logloss: 0.0346303
[857]	valid_0's multi_logloss: 0.0346197
[858]	valid_0's multi_logloss: 0.0346021
[859]	valid_0's multi_logloss: 0.0345916
[860]	valid_0's multi_logloss: 0.0345787
[861]	valid_0's multi_logloss: 0.034567
[862]	valid_0's multi_logloss: 0.0345599
[863]	valid_0's multi_logloss: 0.0345429
[864]	valid_0's multi_logloss: 0.0345363
[865]	valid_0's multi_logloss: 0.034522
[866]	valid_0's multi_logloss: 0.0345086
[867]	valid_0's multi_logloss: 0.0344974
[868]	valid_0's multi_logloss: 0.0344854
[869]	valid_0's multi_logloss: 0.0344663
[870]	valid_0's multi_logloss: 0.0344559
[871]	valid_0's multi_logloss: 0.034439
[872]	valid_0's multi_logloss: 0.0344206
[873]	valid_0's multi_logloss: 0.0344046
[874]	valid_0's multi

...

[988]	valid_0's multi_logloss: 0.0319724
[989]	valid_0's multi_logloss: 0.0319616
[990]	valid_0's multi_logloss: 0.0319511
[991]	valid_0's multi_logloss: 0.0319411
[992]	valid_0's multi_logloss: 0.0319306
[993]	valid_0's multi_logloss: 0.0319177
[994]	valid_0's multi_logloss: 0.0319061
[995]	valid_0's multi_logloss: 0.0318988
[996]	valid_0's multi_logloss: 0.0318897
[997]	valid_0's multi_logloss: 0.0318801
[998]	valid_0's multi_logloss: 0.0318695
[999]	valid_0's multi_logloss: 0.0318563
[1000]	valid_0's multi_logloss: 0.0318486
Did not meet early stopping. Best iteration is:
[1000]	valid_0's multi_logloss: 0.0318486
[1]	valid_0's multi_logloss: 2.48501
Training until validation scores don't improve for 50 rounds
[2]	valid_0's multi_logloss: 2.27598
[3]	valid_0's multi_logloss: 2.1021
[4]	valid_0's multi_logloss: 1.94907
[5]	valid_0's multi_logloss: 1.82075
[6]	valid_0's multi_logloss: 1.70866
[7]	valid_0's multi_logloss: 1.60936
[8]	valid_0's multi_logloss: 1.51986
[9]	valid_0's multi_

[199]	valid_0's multi_logloss: 0.0802816
[200]	valid_0's multi_logloss: 0.0798917
[201]	valid_0's multi_logloss: 0.0795671
[202]	valid_0's multi_logloss: 0.0793403
[203]	valid_0's multi_logloss: 0.0789519
[204]	valid_0's multi_logloss: 0.0785552
[205]	valid_0's multi_logloss: 0.0782004
[206]	valid_0's multi_logloss: 0.0778348
[207]	valid_0's multi_logloss: 0.077529
[208]	valid_0's multi_logloss: 0.077237
[209]	valid_0's multi_logloss: 0.0769645
[210]	valid_0's multi_logloss: 0.0766576
[211]	valid_0's multi_logloss: 0.0763568
[212]	valid_0's multi_logloss: 0.0760562
[213]	valid_0's multi_logloss: 0.0757155
[214]	valid_0's multi_logloss: 0.0753721
[215]	valid_0's multi_logloss: 0.0750565
[216]	valid_0's multi_logloss: 0.0747974
[217]	valid_0's multi_logloss: 0.0744984
[218]	valid_0's multi_logloss: 0.0741855
[219]	valid_0's multi_logloss: 0.073856
[220]	valid_0's multi_logloss: 0.0736248
[221]	valid_0's multi_logloss: 0.0734147
[222]	valid_0's multi_logloss: 0.0731639
[223]	valid_0's mul

[405]	valid_0's multi_logloss: 0.0483781
[406]	valid_0's multi_logloss: 0.0482962
[407]	valid_0's multi_logloss: 0.0482233
[408]	valid_0's multi_logloss: 0.0481282
[409]	valid_0's multi_logloss: 0.0480378
[410]	valid_0's multi_logloss: 0.0479741
[411]	valid_0's multi_logloss: 0.0479
[412]	valid_0's multi_logloss: 0.0478232
[413]	valid_0's multi_logloss: 0.0477151
[414]	valid_0's multi_logloss: 0.0476518
[415]	valid_0's multi_logloss: 0.0475958
[416]	valid_0's multi_logloss: 0.0475509
[417]	valid_0's multi_logloss: 0.0474562
[418]	valid_0's multi_logloss: 0.047402
[419]	valid_0's multi_logloss: 0.0473041
[420]	valid_0's multi_logloss: 0.0472711
[421]	valid_0's multi_logloss: 0.0472042
[422]	valid_0's multi_logloss: 0.047126
[423]	valid_0's multi_logloss: 0.0470684
[424]	valid_0's multi_logloss: 0.0470298
[425]	valid_0's multi_logloss: 0.0469868
[426]	valid_0's multi_logloss: 0.0469255
[427]	valid_0's multi_logloss: 0.04686
[428]	valid_0's multi_logloss: 0.0467918
[429]	valid_0's multi_l

[609]	valid_0's multi_logloss: 0.0388671
[610]	valid_0's multi_logloss: 0.0388373
[611]	valid_0's multi_logloss: 0.038802
[612]	valid_0's multi_logloss: 0.0387763
[613]	valid_0's multi_logloss: 0.0387564
[614]	valid_0's multi_logloss: 0.0387192
[615]	valid_0's multi_logloss: 0.0386847
[616]	valid_0's multi_logloss: 0.0386494
[617]	valid_0's multi_logloss: 0.0386309
[618]	valid_0's multi_logloss: 0.038614
[619]	valid_0's multi_logloss: 0.038588
[620]	valid_0's multi_logloss: 0.0385579
[621]	valid_0's multi_logloss: 0.0385351
[622]	valid_0's multi_logloss: 0.0384939
[623]	valid_0's multi_logloss: 0.0384561
[624]	valid_0's multi_logloss: 0.0384292
[625]	valid_0's multi_logloss: 0.0384057
[626]	valid_0's multi_logloss: 0.0383856
[627]	valid_0's multi_logloss: 0.0383627
[628]	valid_0's multi_logloss: 0.0383173
[629]	valid_0's multi_logloss: 0.0382707
[630]	valid_0's multi_logloss: 0.0382385
[631]	valid_0's multi_logloss: 0.03822
[632]	valid_0's multi_logloss: 0.0381877
[633]	valid_0's multi

[817]	valid_0's multi_logloss: 0.0343472
[818]	valid_0's multi_logloss: 0.034327
[819]	valid_0's multi_logloss: 0.0343111
[820]	valid_0's multi_logloss: 0.0342938
[821]	valid_0's multi_logloss: 0.0342726
[822]	valid_0's multi_logloss: 0.0342538
[823]	valid_0's multi_logloss: 0.0342351
[824]	valid_0's multi_logloss: 0.0342225
[825]	valid_0's multi_logloss: 0.0342049
[826]	valid_0's multi_logloss: 0.0341925
[827]	valid_0's multi_logloss: 0.0341664
[828]	valid_0's multi_logloss: 0.0341508
[829]	valid_0's multi_logloss: 0.0341426
[830]	valid_0's multi_logloss: 0.0341238
[831]	valid_0's multi_logloss: 0.0341063
[832]	valid_0's multi_logloss: 0.0340941
[833]	valid_0's multi_logloss: 0.0340754
[834]	valid_0's multi_logloss: 0.0340623
[835]	valid_0's multi_logloss: 0.034043
[836]	valid_0's multi_logloss: 0.0340276
[837]	valid_0's multi_logloss: 0.0340087
[838]	valid_0's multi_logloss: 0.0340002
[839]	valid_0's multi_logloss: 0.0339911
[840]	valid_0's multi_logloss: 0.0339795
[841]	valid_0's mu

In [108]:
X_test.shape

(782, 9)

In [109]:
# 예측
pred_y_list = []
for models in model:
    pred_y = models.predict_proba(X_test)
    pred_y_list.append(pred_y.reshape(-1,61))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [110]:
# y_pred = gsearch1.predict_proba(X_test)
# y_pred

In [111]:
submission.iloc[:,1:] = pred_ensemble
submission

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000966,6.531339e-05,0.000230,0.000183,0.000738,0.000088,0.000159,0.000022,8.224319e-05,...,0.001072,3.523209e-04,0.001536,0.000234,0.000173,0.000094,0.000355,0.180088,0.000124,0.000097
1,3126,0.000149,2.673327e-05,0.000035,0.000328,0.000022,0.000015,0.000020,0.000046,3.997215e-06,...,0.000034,6.366620e-06,0.000041,0.000009,0.000011,0.000016,0.000004,0.000021,0.000011,0.000096
2,3127,0.000026,9.742446e-01,0.000046,0.000074,0.000024,0.000068,0.000374,0.000157,1.073923e-04,...,0.000017,5.666917e-05,0.000026,0.001529,0.000054,0.001697,0.000004,0.000058,0.001704,0.000026
3,3128,0.000819,5.695652e-04,0.003490,0.000061,0.000095,0.000195,0.000135,0.002436,1.865336e-04,...,0.000167,3.416594e-04,0.000272,0.000290,0.000251,0.000123,0.001410,0.000129,0.000246,0.003966
4,3129,0.000018,9.234865e-07,0.000006,0.000014,0.000003,0.000003,0.000002,0.000003,6.432008e-06,...,0.000002,5.339458e-07,0.000004,0.000002,0.000002,0.000005,0.000008,0.000002,0.000003,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.000158,1.975062e-05,0.000009,0.000014,0.006775,0.001869,0.000023,0.000072,4.252670e-06,...,0.000171,1.288655e-05,0.000054,0.000009,0.000013,0.000318,0.000009,0.000027,0.000039,0.000037
778,3903,0.000004,1.164031e-05,0.000003,0.000008,0.000008,0.000008,0.000026,0.000005,7.152097e-07,...,0.000008,1.990255e-06,0.000003,0.000002,0.000004,0.000039,0.000002,0.000002,0.000004,0.000004
779,3904,0.000035,4.578918e-06,0.000042,0.000059,0.000010,0.000005,0.000009,0.000014,3.861739e-05,...,0.000017,2.553644e-06,0.000032,0.000010,0.000007,0.000005,0.000156,0.000030,0.000010,0.000016
780,3905,0.000057,1.192063e-04,0.000162,0.000041,0.000007,0.000016,0.003637,0.000007,3.099337e-04,...,0.000013,5.518972e-05,0.000075,0.000225,0.000352,0.000090,0.000005,0.000059,0.000105,0.000022


In [112]:
submission.to_csv('baseline_aug7.csv', index=False)